In [ ]:
# | default_exp model
# | hide
import nbdev
from nbdev.showdoc import *

nbdev.nbdev_export()

In [ ]:
# | exporti

from typing import Union
import os

from lovely_numpy import Lo

import numpy as np
from tidygrad.tensor import Tensor
import safetensors
import safetensors.numpy

In [ ]:
#| export

Params_type = Union[dict[str, tuple], str, os.PathLike]

class Model:
    def __init__(self, params: Params_type):

        self.params = {}

        if isinstance(params, dict):
            for name, shape in params.items():
                self.params[name] = Tensor(np.zeros(shape))

        elif isinstance(params, (str, os.PathLike)):
            model = safetensors.safe_open(params, framework="numpy")
            for name in model.keys():
                self.params[name] = Tensor(model.get_tensor(name), name=name)

        else:
            raise TypeError("params must be a dict or a path")

    def __repr__(self):
        return f"Model with params:\n" + "\n".join([f"\t{name}: {param.shape}" for name, param in self.params.items()])

    def save(self, filename: str):
        d = {key: self.params[key].data for key in self.params.keys()}
        safetensors.numpy.save_file(d, filename)


    def requires_grad(self, value):
        for name, param in self.params.items():
            param.requires_grad = value

    def parameter_list(self):
        return list(self.params.values())

